### Install and import the necessary libraries.

In [7]:
# Install python libraries
!pip install gspread &> /dev/null
!pip install gspread-dataframe &> /dev/null

In [98]:
import pandas as pd
import gspread # Python API for Google Sheets.
from google.colab import auth # to autheticate google account
from google.auth import default # to autheticate google account
from gspread_dataframe import set_with_dataframe, get_as_dataframe # To save dataframes to Gsheets

### Connects and authenticates to Google

In [99]:
# Authenticator to be able to access google drive
auth.authenticate_user()
credentials,_ = default() #asigning variable "_" to catch any other things the credentials might have

gconn = gspread.authorize(credentials)

Function spreadsheet_exists checks if the spreadsheet name already exsits

In [100]:
def spreadsheet_exists(spreadsheet_name):
  try:
      gconn.open(spreadsheet_name)
      exists = True
  except gspread.exceptions.SpreadsheetNotFound:
      exists = False
  return exists

### Random dataframe

In [101]:
import numpy as np

# Create a dictionary with 5 columns of random data
data = {
    'Column1': np.random.rand(20),
    'Column2': np.random.randint(1, 100, 20),
    'Column3': np.random.choice(['A', 'B', 'C', 'D'], 20),
    'Column4': np.random.uniform(0, 1, 20),
    'Column5': np.random.randn(20)
}

# Create a pandas DataFrame from the dictionary
sample_df = pd.DataFrame(data)

### Create a google sheet

In [102]:
spreadsheet_name = 'spreadsheet name'
sheet_name = 'sheet name'

if spreadsheet_exists(spreadsheet_name):
  print(f'This spreadsheet already exist')
else:
  gconn.create(spreadsheet_name)
  worksheet = gconn.open(spreadsheet_name).sheet1
  worksheet.update_title(sheet_name)
  worksheet.format(f'A1:Z1', {'textFormat': {'bold': True}})
  set_with_dataframe(worksheet, sample_df)

### Append data to an existing google spreadsheet

In [129]:
spreadsheet_name = 'spreadsheet name'
sheet_name = 'sheet name'

# Opens a file and calculates the number of rows to append the data everytime ir runs.
spreadsheet_file = gconn.open(spreadsheet_name).worksheet(sheet_name)
file_len = len(spreadsheet_file.get_all_values())

# Checks the file len if > 0 appends if less adds also the headers
if file_len > 0:
  set_with_dataframe(spreadsheet_file,sample_df, row = file_len, include_column_header = False)
else:
  set_with_dataframe(spreadsheet_file,sample_df, include_column_header = True)